In [1]:
import numpy as np
import pandas as pd

In [2]:
def _read_observation_file(filename):
    df = pd.read_csv(filename, sep=r"\s+")
    return df


def write_data(df, filename_out):
    """writes the data extracted from input and result files to
    filename_out

    Parameters
    ----------
    df : pd.DataFrame
        data to be written
    filename_out : string
        name of the output file in which the data should be written
    """
    header = "   ".join(df.columns) + "\n"

    with open(filename_out, "w") as f:
        f.write(header)

    # convert the list of dicts to a dataframe
    for col in df.columns[1:]:
        df[col] = df[col].apply(lambda x: f"{float(x):.4E}")

    # fuse the columns of the dataframe
    df_1col = df.apply(lambda row: "   ".join(list(row)) + "\n", 1)

    # save to output file
    with open(filename_out, "a") as f:
        for i in range(len(df_1col)):
            f.write(df_1col.at[i])

In [9]:
mode = "Int"
df = _read_observation_file(f"./Nebula_NGC_7023_{mode}_full.dat")
df.head()

,X_Y,CO_(4-3),CO_(5-4),CO_(6-5),CO_(7-6),CO_(8-7),CO_(9-8),CO_(10-9),CO_(11-10),CO_(12-11),...,HD_112_microns,H2_0-0_S(0),H2_0-0_S(1),H2_0-0_S(2),H2_0-0_S(3),H2_0-0_S(4),H2_0-0_S(5),H2_1-0_S(1),H2_1-0_S(2),H2_2-1_S(1)/1-0_S(1)
0,00_00,2.800000e-08,5.300000e-08,1.100000e-07,2.000000e-07,2.000000e-07,3.100000e-07,2.500000e-07,3.500000e-07,2.700000e-07,...,7.500000e-09,1.300000e-07,9.100000e-07,9.700000e-07,0.000001,5.900000e-07,9.200000e-07,2.100000e-07,7.600000e-08,0.29


In [10]:
df["X"] = df["X_Y"].apply(lambda x: x.split("_")[0])
df["Y"] = df["X_Y"].apply(lambda x: x.split("_")[1])
df = df.drop(["X_Y"], 1)
df = df.astype(float)
df[["X", "Y"]] = df[["X", "Y"]].astype(int)
df = df.set_index(["X", "Y"])
df.head()

/tmp/ipykernel_519743/687504521.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop(["X_Y"], 1)


,,CO_(4-3),CO_(5-4),CO_(6-5),CO_(7-6),CO_(8-7),CO_(9-8),CO_(10-9),CO_(11-10),CO_(12-11),CO_(13-12),...,HD_112_microns,H2_0-0_S(0),H2_0-0_S(1),H2_0-0_S(2),H2_0-0_S(3),H2_0-0_S(4),H2_0-0_S(5),H2_1-0_S(1),H2_1-0_S(2),H2_2-1_S(1)/1-0_S(1)
X,Y,,,,,,,,,,,,,,,,,,,,,
0,0,2.800000e-08,5.300000e-08,1.100000e-07,2.000000e-07,2.000000e-07,3.100000e-07,2.500000e-07,3.500000e-07,2.700000e-07,2.400000e-07,...,7.500000e-09,1.300000e-07,9.100000e-07,9.700000e-07,0.000001,5.900000e-07,9.200000e-07,2.100000e-07,7.600000e-08,0.29


In [11]:
# conversion from W m$^{-2}$ sr$^{-1}$   to  erg cm$^{-2}$ s$^{-1}$ sr$^{-1}$
df *= 1e3

In [12]:
df = df.rename(
    columns={
        "CO_(4-3)": "co_v0_j4__v0_j3",
        "CO_(5-4)": "co_v0_j5__v0_j4",
        "CO_(6-5)": "co_v0_j6__v0_j5",
        "CO_(7-6)": "co_v0_j7__v0_j6",
        "CO_(8-7)": "co_v0_j8__v0_j7",
        "CO_(9-8)": "co_v0_j9__v0_j8",
        "CO_(10-9)": "co_v0_j10__v0_j9",
        "CO_(11-10)": "co_v0_j11__v0_j10",
        "CO_(12-11)": "co_v0_j12__v0_j11",
        "CO_(13-12)": "co_v0_j13__v0_j12",
        "CO_(15-14)": "co_v0_j15__v0_j14",
        "CO_(16-15)": "co_v0_j16__v0_j15",
        "CO_(17-16)": "co_v0_j17__v0_j16",
        "CO_(18-17)": "co_v0_j18__v0_j17",
        "CO_(19-18)": "co_v0_j19__v0_j18",
        "13CO_(5-4)": "13c_o_j5__j4",
        "13CO_(6-5)": "13c_o_j6__j5",
        "13CO_(7-6)": "13c_o_j7__j6",
        "13CO_(8-7)": "13c_o_j8__j7",
        "13CO_(9-8)": "13c_o_j9__j8",
        "13CO_(10-9)": "13c_o_j10__j9",
        "C18O_(5-4)": "c_18o_j5__j4",
        "C18O_(9-8)": "c_18o_j9__j8",
        "CH+_(1-0)": "chp_j1__j0",
        "CH+_(2-1)": "chp_j2__j1",
        "CH+_(3-2)": "chp_j3__j2",
        "HCO+_(1-0)": "hcop_j1__j0",
        "HCO+_(6-5)": "hcop_j6__j5",
        "C+": "cp_el2p_j3_2__el2p_j1_2",
        "C_609_microns": "c_el3p_j1__el3p_j0",
        "C_370_microns": "c_el3p_j2__el3p_j1",
        "O_145_microns": "o_el3p_j0__el3p_j1",
        "O_63_microns": "o_el3p_j1__el3p_j2",
        "HD_112_microns": "hd_v0_j1__v0_j0",
        "H2_0-0_S(0)": "h2_v0_j2__v0_j0",
        "H2_0-0_S(1)": "h2_v0_j3__v0_j1",
        "H2_0-0_S(2)": "h2_v0_j4__v0_j2",
        "H2_0-0_S(3)": "h2_v0_j5__v0_j3",
        "H2_0-0_S(4)": "h2_v0_j6__v0_j4",
        "H2_0-0_S(5)": "h2_v0_j7__v0_j5",
        "H2_1-0_S(1)": "h2_v1_j3__v0_j1",
        "H2_1-0_S(2)": "h2_v1_j4__v0_j2",
        # "H2_2-1_S(1)/1-0_S(1)": "",
    }
)
df = df.drop("H2_2-1_S(1)/1-0_S(1)", 1)

/tmp/ipykernel_519743/244354228.py:48: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop("H2_2-1_S(1)/1-0_S(1)", 1)


In [13]:
# write_data(df, "./Nebula_NGC_7023_Err_fit.dat")

In [14]:
df.to_pickle(f"./Nebula_NGC_7023_{mode}.pkl")

In [1]:
import yaml

with open(f"input_params.yaml") as f:
    params = yaml.safe_load(f)

In [6]:
import numpy as np

np.isfinite(params["run_params"]["mcmc"]["regu_spatial_N0"])

False